<a href="https://colab.research.google.com/github/yfb2022/STAT_DL_INTRO/blob/main/6_1_one_hot_encoding_of_words_or_characters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
keras.__version__

'2.15.0'

# One-hot encoding of words or characters

원-핫 인코딩은 토큰을 벡터로 변환하는 가장 일반적이고 기본적인 방법.

이는 모든 단어에 고유한 정수 인덱스를 연결한 다음 이 정수 인덱스 i를 어휘 크기인 N 크기의 이진 벡터로 변환하는 것으로 구성됩니다. 이는 i 번째 항목인 1을 제외하고 모두 0이 됩니다. .

물론 문자 수준에서도 원-핫 인코딩을 수행할 수 있습니다.

하나는 단어용이고 다른 하나는 문자용입니다


Word level one-hot encoding (toy example):

In [2]:
import numpy as np

# This is our initial data; one entry per "sample"
# (in this toy example, a "sample" is just a sentence, but
# it could be an entire document).
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# First, build an index of all tokens in the data.
token_index = {}
# token_index <- list()
for sample in samples:
    # We simply tokenize the samples via the `split` method.
    # in real life, we would also strip punctuation and special characters
    # from the samples.
    for word in sample.split():
        if word not in token_index:
            # Assign a unique index to each unique word
            token_index[word] = len(token_index) + 1
            # Note that we don't attribute index 0 to anything.

# for (sample in samples)
#    for (word in strsplit(sample, " ")[[1]])
#        if (!word %in% names(token_index))
#           token_index[[word]] =  length(token_index) + 2

# Next, we vectorize our samples.
# We will only consider the first `max_length` words in each sample.
max_length = 10

# This is where we store our results:
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
# results <- array(0, dim = c(length(samples), max_length, max(as.integer(token_index))))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

# for (i in 1:length(samples)) {
#     sample <- samples[[i]]
#     words <- head(strsplit(sample, " ")[[1]], n = max_length)
#     for (j in 1:length(words)) {
#         index <- token_index[[words[[j]]]]
#         results[[i, j, index]] <- 1
#     }
# }

Character level one-hot encoding (toy example)

In [3]:
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable  # All printable ASCII characters.
# ascii_tokens <- c("", sapply(as.raw(c(32:126)), rawToChar))
token_index = dict(zip(characters, range(1, len(characters) + 1)))
# token_index <- c(1: (length(ascii_tokens)))
# names(token_index) <- ascii_tokens
max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.

원핫 인코딩 Keras 라이브러리

Using Keras for word-level one-hot encoding:

In [4]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# We create a tokenizer, configured to only take
# into account the top-1000 most common words
tokenizer = Tokenizer(num_words=1000)
# This builds the word index
tokenizer.fit_on_texts(samples)

# This turns strings into lists of integer indices.
sequences = tokenizer.texts_to_sequences(samples)

# You could also directly get the one-hot binary representations.
# Note that other vectorization modes than one-hot encoding are supported!
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# This is how you can recover the word index that was computed
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.



원-핫 인코딩의 변형은 소위 "원-핫 해싱 트릭"입니다. 이는 어휘의 고유 토큰 수가 너무 커서 명시적으로 처리할 수 없을 때 사용할 수 있습니다.

각 단어에 인덱스를 명시적으로 할당하고 사전에 이러한 인덱스에 대한 참조를 유지하는 대신 단어를 고정된 크기의 벡터로 해시할 수 있습니다.이는 일반적으로 매우 가벼운 해싱 함수를 사용하여 수행됩니다.

이 방법의 가장 큰 장점은 명시적인 단어 인덱스를 유지하지 않아도 되므로 메모리가 절약되고 데이터의 온라인 인코딩이 가능하다는 것입니다(사용 가능한 모든 데이터를 보기 전에 즉시 토큰 벡터 생성 시작). 이 방법의 한 가지 단점은 "해시 충돌"에 취약하다는 것입니다. 두 개의 서로 다른 단어가 동일한 해시로 끝날 수 있으며 결과적으로 이러한 해시를 보는 모든 기계 학습 모델은 이러한 단어 간의 차이를 구분할 수 없습니다. 해시 공간의 차원이 해시되는 고유 토큰의 총 수보다 훨씬 클 경우 해시 충돌 가능성이 감소합니다.

Word-level one-hot encoding with hashing trick (toy example):

In [5]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# We will store our words as vectors of size 1000.
# Note that if you have close to 1000 words (or more)
# you will start seeing many hash collisions, which
# will decrease the accuracy of this encoding method.
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # Hash the word into a "random" integer index
        # that is between 0 and 1000
        index = abs(hash(word)) % dimensionality
        # index <- abs(spooky.32(words[[i]])) %% dimensionality
        results[i, j, index] = 1.